In [ ]:
cfg_var_SSH.OBS_path_load(cfg_var_SSH.var)
cfg_var_SSH.OBS_ds = xr.open_mfdataset(cfg_var_SSH.OBS_file_list[0][0], 
                       chunks={'time': 12}, 
                       parallel=True,
                       decode_cf=True, 
                       decode_times=True,)
cfg_var_SSH.OBS_ds = cfg_var_SSH.OBS_ds.rename({cfg_var_SSH.OBS_var: cfg_var_SSH.var})

if cfg_var_SSH.var=='SSH':
    cfg_var_SSH.OBS_ds=cfg_var_SSH.OBS_ds.assign_coords(time=cfg_var_SSH.ODA_ds.time[396:732])
elif cfg_var_SSH.var=='photoC_TOT_zint_100m':
    cfg_var_SSH.OBS_ds=cfg_var_SSH.OBS_ds.assign_coords(time=cfg_var_SSH.ODA_ds.time[456:732])
cfg_var_SSH.OBS_ds.time

# global mean removal for SSH
if cfg_var_SSH.var=='SSH':
    lat_mask = (ds_grid.TLAT >= -60) & (ds_grid.TLAT <= 60)

    area_selected = ds_grid.TAREA.where(lat_mask, drop=True)  # 선택된 지역의 면적
    ssh_selected = cfg_var_SSH.OBS_ds[cfg_var_SSH.var].where(lat_mask, drop=True)  # 선택된 지역의 SSH 데이터

    # 필터링된 지역에 대해 global mean 계산
    cfg_var_SSH.OBS_ds['gm'] = (ssh_selected * area_selected).sum(dim=['nlat', 'nlon']) / area_selected.sum(dim=['nlat', 'nlon'])
    
    # 원 데이터에서 global mean을 제거한 값
    cfg_var_SSH.OBS_ds['SSH'] = cfg_var_SSH.OBS_ds[cfg_var_SSH.var] - cfg_var_SSH.OBS_ds['gm']


# Step 1: 결측값이 아닌 값의 비율 계산
# 결측값이 아닌 값들에 대해 True, 결측값에 대해 False로 표시
valid_data_count = (~cfg_var_SSH.OBS_ds['photoC_TOT_zint_100m'].isnull()).sum(dim='time')
# Step 2: 전체 시간 축의 길이 (즉, 데이터가 있어야 할 총 시간 개수)
total_time_steps = cfg_var_SSH.OBS_ds['time'].size
# Step 3: 80% 이상의 값이 있는지 계산
# 비율 계산 후, 80% 이상이면 True, 그렇지 않으면 False로 표시
threshold = 0.8
mask = (valid_data_count / total_time_steps) >= threshold
# Step 4: 마스크 변수를 데이터셋에 추가
cfg_var_SSH.OBS_ds['mask_80_percent'] = mask
cfg_var_SSH.OBS_ds['mask_80_percent'].compute()